In [34]:
#Importamos las librerías
import cv2
import os
import numpy as np 
import keras
import matplotlib.pyplot as plt
import glob

In [35]:
# Descomprimimos el archivo
!tar -xzf "archive.zip" -C "./"

Chessman-image-dataset/Chess/Bishop/00000000.JPG: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000001.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000002.JPG: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000003.png: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000004.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000006.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000007.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000008.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000009.png: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000010.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/00000011.jpg: Can't restore time: Invalid argument
Chessman-image-dataset/Chess/Bishop/0000001

In [36]:
#Hacemos una relación entre los nombres de las imágenes y las clases
MAP_CHARACTERS = { 0: 'Bishop', 1: 'King', 2: 'Knight', 3: 'Pawn', 4: 'Queen', 5: 'Rook' }
# Vamos a standarizar todas las imágenes a tamaño 64x64
IMG_SIZE = 64

In [37]:
def load_train_set(dirname, map_characters, verbose=True):
	"""Esta función carga los datos de training en imágenes.

	Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
	para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.

	Args:
		dirname: directorio completo del que leer los datos
		map_characters: variable de mapeo entre labels y personajes
		verbose: si es True, muestra información de las imágenes cargadas

	Returns:
		X, y: X es un array con todas las imágenes cargadas con tamaño
	IMG SIZE x IMG SIZE
				y es un array con las labels de correspondientes a cada imagen
	"""
	X_train = []
	y_train = []
	for label, character in map_characters.items():        
		files = os.listdir(os.path.join(dirname, character))
		images = [file for file in files if file.endswith("jpg")]
		if verbose:
			print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
		for image_name in images:
			image = cv2.imread(os.path.join(dirname, character, image_name))
			X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
			y_train.append(label)
	return np.array(X_train), np.array(y_train)

def load_test_set(dirname, map_characters, verbose=True):
   """Esta función funciona de manera equivalente a la función load_train_set
   pero cargando los datos de test."""
   X_test = []
   y_test = []
   reverse_dict = {v: k for k, v in map_characters.items()}
   for filename in glob.glob(dirname + '/*.*'):
       char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
       if char_name in reverse_dict:
           image = cv2.imread(filename)
           image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
           X_test.append(image)
           y_test.append(reverse_dict[char_name])
   if verbose:
       print("Leídas {} imágenes de test".format(len(X_test)))
   return np.array(X_test), np.array(y_test)
   

In [38]:
import shutil
from sklearn.model_selection import train_test_split


def split_dataset(source_dir, dest_dir, split_ratio=0.3):
	"""
	Esta función divide el dataset en dos partes, una para training y otra para test, y lo pone en directorios distintos.
	"""
	if not os.path.exists(dest_dir):
		os.makedirs(dest_dir)
	
	for character in os.listdir(source_dir):
		character_path = os.path.join(source_dir, character)
		if os.path.isdir(character_path):
			images = [file for file in os.listdir(character_path) if file.endswith("jpg")]
			train_images, test_images = train_test_split(images, test_size=split_ratio, random_state=42)
			
			character_dest_path = os.path.join(dest_dir, character)
			if not os.path.exists(character_dest_path):
				os.makedirs(character_dest_path)
			
			for image in test_images:
				shutil.move(os.path.join(character_path, image), os.path.join(character_dest_path, image))

source_directory = ".\\Chessman-image-dataset\\Chess"
destination_directory = ".\\Chessman-image-dataset\\Chess_test"
split_dataset(source_directory, destination_directory)

In [39]:
# Cargamos los datos. Si no estás trabajando en colab, cambia los paths por
# los de los ficheros donde hayas descargado los datos.
DATASET_TRAIN_PATH_COLAB = ".\\Chessman-image-dataset\\Chess"
DATASET_TEST_PATH_COLAB = "/root/.keras/datasets/simpsons_testset"
X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_t, y_t = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)# Vamos a barajar aleatoriamente los datos. Esto es importante ya que si no
# lo hacemos y, por ejemplo, cogemos el 20% de los datos finales como validation
# set, estaremos utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

Leyendo 49 imágenes encontradas de Bishop
Leyendo 42 imágenes encontradas de King
Leyendo 65 imágenes encontradas de Knight
Leyendo 60 imágenes encontradas de Pawn
Leyendo 46 imágenes encontradas de Queen
Leyendo 60 imágenes encontradas de Rook
Leídas 0 imágenes de test
